In [1]:
import pandas as pd
import numpy as np
from linestar import get_proj_data, get_historical_data, OpponentTeams
import cvxpy

rng = np.random.default_rng()

In [2]:
#data = get_proj_data()

In [3]:
slate = get_historical_data("2022-05-19")

/home/eadains/DFS/linestar.py:53: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 53 of the file /home/eadains/DFS/linestar.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


# Correlation Matrix

In [4]:
bat_ord_corr = pd.read_csv("./data/batting_order_corr.csv", index_col="Order")
bat_ord_corr.columns = bat_ord_corr.columns.astype(float)
opp_pitcher_corr = pd.read_csv("./data/opp_pitcher_corr.csv", index_col=0)
opp_pitcher_corr = opp_pitcher_corr.loc["Scored", "Scored Opposing"]

corr = pd.DataFrame(columns=slate["Player"], index=slate["Player"], dtype=float)

In [5]:
for row in slate.itertuples():
    # Correlation with themselves is 1
    corr.loc[row.Player, row.Player] = 1

    # If pitcher, set correlation to everyone else to 0
    if row.Position == "P":
        corr.loc[row.Player, corr.columns != row.Player] = 0

    else:
        # Setting correlation to other batters on the same team according to
        # batting order
        for teammate in slate.loc[slate["Team"] == row.Team, :].itertuples():
            # If the teammate is the pitcher, then 0 correlation
            if teammate.Position == "P":
                corr.loc[row.Player, teammate.Player] = 0
                corr.loc[teammate.Player, row.Player] = 0
            else:
                order_corr = bat_ord_corr.loc[row.Order, teammate.Order]
                corr.loc[row.Player, teammate.Player] = order_corr
                corr.loc[teammate.Player, row.Player] = order_corr

        # Set correlation to opposing pitcher
        corr.loc[row.Player, row.Opp_Pitcher] = opp_pitcher_corr
        corr.loc[row.Opp_Pitcher, row.Player] = opp_pitcher_corr
        # Correlations with every other player is 0
        corr.loc[row.Player, corr.loc[row.Player].isna()] = 0

In [6]:
# Check that the correlation matrix is symmetric and all its eigvenvalues are >= 0
# These two conditions jointly imply the matrix is positive semi-definite
np.array_equal(corr, corr.T) & np.all(np.linalg.eigvals(corr) >= 0)

True

# Standard Deviation

In [7]:
historical = pd.read_csv("./data/linestar_data.csv")
hist_std = historical.groupby("Player").std()["Scored"]

# Default standard deviations for players with missing values
default_pitcher_std = 15
default_other_std = 10

# Get historical standard deviation of scored points for players
# on the current slate
hist_std = hist_std.loc[slate["Player"]]

for player in hist_std[hist_std.isna()].index:
    player_position = slate.loc[slate["Player"] == player, "Position"].values[0]
    if player_position == "P":
        hist_std.loc[player] = default_pitcher_std
    else:
        hist_std.loc[player] = default_other_std

# Score Distribution

In [8]:
mean = slate["Consensus"]
cov = np.diag(hist_std) @ corr @ np.diag(hist_std)

opps = OpponentTeams(slate)

In [9]:
results = []
for x in range(100):
    sample = rng.multivariate_normal(mean, cov)
    order_stat = opps.get_order_stat(sample, 50, 99)
    results.append((sample, order_stat))

In [10]:
mu_g = np.mean([x[1] for x in results])
var_g = np.var([x[1] for x in results])

cov_g = []
for n in range(len(mean)):
    order_stat_cov = np.cov([x[0][n] for x in results], [x[1] for x in results])
    cov_g.append(order_stat_cov[0, 1])
cov_g = np.array(cov_g)

# Optimization

In [11]:
pos_mat = opps.pos_mat
teams = opps.teams
games = opps.games

In [37]:
selection = cvxpy.Variable(len(slate), boolean=True)
teams_var = cvxpy.Variable(len(teams.columns), boolean=True)
games_var = cvxpy.Variable(len(games.columns), boolean=True)
lambda_param = cvxpy.Parameter(nonneg=True)

# Total salary must be less than or equal to $35,000
salary = selection @ slate["Salary"] <= 35000

# Must select players from at least 3 different teams
teams_var_con = teams_var <= selection @ teams
teams_con = cvxpy.sum(teams_var) >= 3

# Must select players from at least 2 different games
games_var_con = games_var <= selection @ games
games_con = cvxpy.sum(games_var) >= 2

# No more than 4 players, not counting the pitcher, can be selected from the same team
# First term is our selected players multiplied by a boolean array where 1's indicate non-pitcher players.
# This filters the selected players so the constraint only applies to non-pitcher players
players_teams = cvxpy.multiply(selection, (~pos_mat["P"].astype(bool)).astype(int)) @ teams <= 4

# Must have 9 players selected
total_players = cvxpy.sum(selection) == 9

# Max and min number of players we can select for each position
# Must always have 1 pitcher, who cannot fill the UTIL position
# We can select up to 1 additional player from each other position because
# the second can fill the UTIL position
positions_max = [1, 2, 2, 2, 2, 4]
positions_min = [1, 1, 1, 1, 1, 3]
positions_max_con = selection @ pos_mat <= positions_max
positions_min_con = selection @ pos_mat >= positions_min

mu_w = selection @ slate["Consensus"] - mu_g
sigma_w = cvxpy.quad_form(selection, cov) + var_g - 2 * selection @ cov_g
obj = mu_w - lambda_param * sigma_w

In [48]:
constraints = [salary,
               teams_var_con,
               teams_con,
               games_var_con,
               games_con,
               players_teams,
               total_players,
               positions_max_con,
               positions_min_con]
problem = cvxpy.Problem(cvxpy.Maximize(obj), constraints=constraints)
problem.solve(solver=cvxpy.SCIP, verbose=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 26 03:22:38 PM: Your problem has 69 variables, 9 constraints, and 1 parameters.
(CVXPY) May 26 03:22:38 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 03:22:38 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 03:22:38 PM: Compiling problem (target solver=SCIP).
(CVXPY) May 26 03:22:38 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIP
(CVXPY) May 26 03:22:38 PM: Applying reduction FlipObjective
(CVXPY) May 26 03:22:38 PM: Applying reduction D

KeyboardInterrupt: 

379k |  24 | 132 | 192 |  99 | 161 |  1 | 106 | 158 |-1.403487e+02 |-1.115615e+02 |  25.80%|   5.23%
  0.2s|   600 |   443 |  1725 |   2.8 |  4446k |  24 | 132 | 205 | 101 | 162 |  1 | 120 | 169 |-1.403487e+02 |-1.115615e+02 |  25.80%|   5.36%
  0.3s|   700 |   515 |  1908 |   2.7 |  4574k |  27 | 132 | 217 |  99 | 187 |  1 | 132 | 172 |-1.403487e+02 |-1.115615e+02 |  25.80%|   5.42%
  0.3s|   800 |   589 |  2086 |   2.6 |  4607k |  29 | 132 | 226 |  99 | 203 |  1 | 143 | 177 |-1.403487e+02 |-1.115615e+02 |  25.80%|   5.46%
  0.3s|   900 |   677 |  2276 |   2.5 |  4607k |  29 | 132 | 232 |  99 | 208 |  1 | 159 | 187 |-1.401608e+02 |-1.115615e+02 |  25.64%|   6.01%
  0.3s|  1000 |   751 |  2403 |   2.4 |  4622k |  29 | 132 | 238 |  99 | 209 |  1 | 180 | 192 |-1.401608e+02 |-1.115615e+02 |  25.64%|   6.03%
* 0.3s|  1074 |   797 |  2551 |   2.3 |    LP  |  29 | 132 | 244 |  99 | 219 |  1 | 196 | 198 |-1.401608e+02 |-1.116353e+02 |  25.55%|   7.38%
  0.3s|  1100 |   817 |  2623 |   2.4 |  